In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import pandas as pd
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from tqdm import tqdm
import tensorflow as tf

2023-11-24 00:07:04.253730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [4]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [5]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [6]:
all_games = [d[1] for d in allHours]

In [7]:
user_games = defaultdict(set)
for u,i,d in allHours:
    user_games[u].add(i)

In [8]:
# Construct negative sample randomly for validation
hoursValid_with_negatives = []
labels = []
for userID,itemID,d in hoursValid:
    positive_game_id = itemID
    negative_game_candidates = list(set(all_games) - user_games[userID])
    
    if negative_game_candidates:
        negative_game_id = random.choice(negative_game_candidates)
        
        hoursValid_with_negatives.append((userID,itemID,d))
        labels.append(1)

        hoursValid_with_negatives.append((userID, negative_game_id, {}))
        labels.append(0)
        
labels = np.array(labels)

In [9]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [10]:
ug_train = defaultdict(set)
gu_train = defaultdict(set)

for u,g,_ in hoursTrain:
    ug_train[u].add(g)
    gu_train[g].add(u)

In [11]:
### Bayesian Personalized Ranking (Tensorflow)

In [12]:
# Mapping from userID and itemID to indices
userIDs = {}
itemIDs = {}
interactions = []
for u, g, d in hoursTrain:
    r = d['hours_transformed']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in itemIDs: itemIDs[g] = len(itemIDs)
    interactions.append((u,g,r))
    
random.shuffle(interactions)

nTrain = int(len(interactions)*0.9)
nTest = len(interactions)- nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)

for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)


In [13]:
items = list(itemIDs.keys())
class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb

    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui

    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

In [14]:
def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [], [], []
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in itemsPerUser[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])

        loss = model(sampleU,sampleI,sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [15]:
def predict_would_play(u,i):
    user_id = userIDs.get(u, None)
    item_id = itemIDs.get(i, None)
    
    if user_id is not None and item_id is not None:
        score = modelBPR.predict(user_id,item_id).numpy()
        return score
    else:
        # default score for unseen user or id
        return 0

In [16]:
LR = 0.1
K = 5
lamb = 1e-5
iterations = 100

In [17]:
optimizer = tf.keras.optimizers.Adam(LR)
modelBPR = BPRbatch(K, lamb)
for i in range(iterations):
    obj = trainingStepBPR(modelBPR, interactions)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))


2023-11-24 00:07:40.271410: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10398 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:85:00.0, compute capability: 6.1
2023-11-24 00:07:55.295749: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b3516f82c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-24 00:07:55.295793: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2023-11-24 00:07:55.303942: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-24 00:07:55.815467: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-24 00:07:56.058021: I ./tensorflow/compiler/jit/device_compiler.h:186] C

iteration 10, objective = 0.51601505
iteration 20, objective = 0.48353913
iteration 30, objective = 0.46613118
iteration 40, objective = 0.458682
iteration 50, objective = 0.45653126
iteration 60, objective = 0.45299065
iteration 70, objective = 0.45042548
iteration 80, objective = 0.44632196
iteration 90, objective = 0.44679296
iteration 100, objective = 0.4433291


In [18]:
all_scores = defaultdict(list)
for u,i,d in tqdm(hoursValid_with_negatives):
    pred = predict_would_play(u,i)
    all_scores[u].append((pred,i))

100%|██████████| 19998/19998 [00:50<00:00, 392.93it/s]


In [19]:
# Testing on the set I created earlier with the negative samples
all_scores = {u:sorted(l,reverse=True) for u, l in all_scores.items()}
top_half_scores = {}
for key, lst in all_scores.items():
    mu = np.mean([p[0] for p in lst])
    
    # Find the breakpoint
    index = next((i for i, p in enumerate(lst) if p[0] <= mu), len(lst))
    top_half_scores[key] = {p[1] for p in lst[:index]}
    
predictionss = []
for u, g, d in tqdm(hoursValid_with_negatives):
    if g in top_half_scores[u]:
        # Predict 1 for items with score above the mean
        predictionss.append(1)
    else:
        # Predict 0 otherwise
        predictionss.append(0)


100%|██████████| 19998/19998 [00:00<00:00, 1572952.49it/s]


In [20]:
# Accuracy
acc = np.mean(predictionss == labels)
print(f'Accuracy of BPR with learning_rate={LR}, K={K}, and lambda={lamb} is {acc}')

Accuracy of BPR with learning_rate=0.1, K=5, and lambda=1e-05 is 0.7494749474947495


In [21]:
predictions = open("predictions_Played.csv", 'w')

In [22]:
all_scores = defaultdict(list)
test_set = []
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    p = predict_would_play(u,g)
    all_scores[u].append((p,g))
    test_set.append((u,g,{}))

In [23]:
all_scores = {u:sorted(l,reverse=True) for u, l in all_scores.items()}
top_half_scores = {}
for key, lst in all_scores.items():
    mu = np.mean([p[0] for p in lst])
    index = next((i for i, p in enumerate(lst) if p[0] <= mu), len(lst))
    top_half_scores[key] = {p[1] for p in lst[:index]}
    

pred = 0
for u, g, d in tqdm(test_set):
    if g in top_half_scores[u]:
        pred = 1
    else:
        pred = 0
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')
predictions.close()

100%|██████████| 20000/20000 [00:00<00:00, 767491.74it/s]


In [24]:
##################################################
# Hours played prediction                        #
##################################################

In [25]:
def MSE(y, ypred):
    differences = [(x-y)**2 for x,y in zip(ypred,y)]
    return sum(differences) / len(differences)

In [26]:
trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

In [27]:
hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)
for u,g,d in allHours:
    r = d['hours_transformed']
    hoursPerUser[u].append((g,r))
    hoursPerItem[g].append((u,r))

In [28]:
trainHours = [r[2]['hours_transformed'] for r in allHours]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

In [29]:
globalAverage

3.717863918924211

In [30]:
per = int(len(allHours) * 0.9)
Train = allHours[:per]
Valid = allHours[per:]
assert (len(Train) + len(Valid)) == len(allHours)

In [31]:
validMSE = 0
for u,g,d in hoursValid:
    r = d['hours_transformed']
    se = (r - globalAverage)**2
    validMSE += se

validMSE /= len(hoursValid)

print("Validation MSE (average only) = " + str(validMSE))

Validation MSE (average only) = 5.315913624424781


In [32]:
def iterate(lamb):
    newAlpha = 2.5
    for u,g,d in allHours:
        r = d['hours_transformed']
        newAlpha += r - (betaU[u] + betaI[g])
    alpha = newAlpha / len(allHours)
    for u in hoursPerUser:
        newBetaU = 0
        for g,r in hoursPerUser[u]:
            newBetaU += r - (alpha + betaI[g])
        betaU[u] = newBetaU / (lamb + len(hoursPerUser[u]))
    for g in hoursPerItem:
        newBetaI = 0
        for u,r in hoursPerItem[g]:
            newBetaI += r - (alpha + betaU[u])
        betaI[g] = newBetaI / (lamb + len(hoursPerItem[g]))
    mse = 0
    for u,g,d in allHours:
        r = d['hours_transformed']
        prediction = alpha + betaU[u] + betaI[g]
        mse += (r - prediction)**2
    regularizer = 0
    for u in betaU:
        regularizer += betaU[u]**2
    for g in betaI:
        regularizer += betaI[g]**2
    mse /= len(allHours)
    return mse, mse + lamb*regularizer, alpha

In [33]:
lamb = 5
betaU = {}
betaI = {}
for u in hoursPerUser:
    betaU[u] = 0

for g in hoursPerItem:
    betaI[g] = 0
mse,objective,alpha = iterate(lamb)
newMSE,newObjective,alpha = iterate(lamb)
iterations = 2
while iterations < 10 or objective - newObjective > 0.05:
    mse, objective = newMSE, newObjective
    newMSE, newObjective, alpha = iterate(lamb)
    iterations += 1
    if iterations % 20 == 0:
        print("Objective after "
            + str(iterations) + " iterations = " + str(newObjective))
        print("MSE after "
            + str(iterations) + " iterations = " + str(newMSE))
    if abs(newMSE - mse) < 0.00001:
        print(f"Converged at iteration {iterations}")
        print(f"Converged with oldMSE = {mse}")
        print(f"Converged with newMSE = {newMSE}")
        break

Objective after 20 iterations = 23872.187687423386
MSE after 20 iterations = 2.784333973113838
Objective after 40 iterations = 23673.150046572333
MSE after 40 iterations = 2.7835177440376486


In [34]:
validMSE = 0
for u,g,d in hoursValid:
    r = d['hours_transformed']
    bu = 0
    bi = 0
    if u in betaU:
        bu = betaU[u]
    if g in betaI:
        bi = betaI[g]
    prediction = alpha + bu + bi
    validMSE += (r - prediction)**2
ß
validMSE /= len(hoursValid)
print("Validation MSE = " + str(validMSE))

Validation MSE = 2.74830185348576


In [35]:
predictions = open("predictions_Hours.csv", 'w')
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    # Logic...
    bu = 0
    bi = 0
    if u in betaU:
        bu = betaU[u]
    if g in betaI:
        bi = betaI[g]
        
    _ = predictions.write(u + ',' + g + ',' + str(alpha + bu + bi) + '\n')

predictions.close()

In [36]:
!jupyter nbconvert --to script Bayesian_Personalized_ranking.ipynb --output 'BPR'

[NbConvertApp] Converting notebook Bayesian_Personalized_ranking.ipynb to script
[NbConvertApp] Writing 11159 bytes to BPR.py
